In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
from data_split import read_json_from_local, prepare_data, split_data
import matplotlib.pyplot as plt

In [ ]:
# [Keep the data loading, preparation, and model training code as it was]
# ...

In [ ]:
# Function to make predictions using locally saved models
def predict_local(input_data):
    predictions = {}
    for target in targets:
        model = joblib.load(os.path.join(model_path, f"{target}_model.joblib"))
        predictions[target] = model.predict(input_data)
    return pd.DataFrame(predictions)

# Function to get actual and predicted values for a given date
def get_actual_and_predicted(date, prepared_data, models):
    # Convert date to datetime if it's not already
    date = pd.to_datetime(date)
    
    # Prepare input data for prediction
    input_data = pd.DataFrame({
        'day_of_week': [date.dayofweek],
        'month': [date.month],
        'is_weekday': [1 if date.dayofweek < 5 else 0]
    })
    
    # Make predictions
    predictions = predict_local(input_data)
    
    # Get actual values if available
    actual_data = prepared_data[prepared_data['date'] == date]
    
    results = {}
    for target in targets:
        results[target] = {
            'predicted': predictions[target].values[0],
            'actual': actual_data[target].values[0] if not actual_data.empty else 'N/A'
        }
    
    return results

In [ ]:
# Interactive date input and prediction
while True:
    date_str = input("Enter a date (YYYY-MM-DD) to predict for, or 'q' to quit: ")
    if date_str.lower() == 'q':
        break
    
    try:
        date = pd.to_datetime(date_str)
        results = get_actual_and_predicted(date, prepared_data, models)
        
        print(f"\nPredictions and Actual Values for {date.date()}:")
        for target, values in results.items():
            print(f"{target}:")
            if 'file_count' in target:
                print(f"  Predicted: {int(round(values['predicted']))}")
                print(f"  Actual: {int(values['actual']) if values['actual'] != 'N/A' else 'N/A'}")
            else:  # for total_records
                print(f"  Predicted: {int(round(values['predicted']):,)}")
                print(f"  Actual: {int(values['actual']):,} if values['actual'] != 'N/A' else 'N/A'}")
            print()
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")